In [1]:
## load necessary packages
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pyhdfe

In [2]:
## load industry types, see online supplementary document of the paper
indtype = pd.read_csv('processdb/indtype.tsv',sep='\t',dtype='str')
indtype.columns=['ind','indtype']
indtype

,ind,indtype
0,011,geo
1,013,geo
2,016,geo
3,017,geo
4,018,geo
...,...,...
410,964,public
411,965,public
412,966,public
413,971,public


## Load region-ind data

In [3]:
## load data and create indicators and growth rate
def load_ind_vec(var,ind='sic'):
    indvecdf = pd.read_parquet(f"processdb/msa_{ind}.parquet",columns=['CBSAFP',ind,f'num_{var}_2011',f'num_{var}_2019']) ## defaults to all
    indvecdf = indvecdf[(indvecdf[ind]!='9999')].rename(columns={'CBSAFP':'region',f'num_{var}_2011':'raw',f'num_{var}_2019':'after8'})
    indvecdf['ind'] = indvecdf[ind].str[:3]
    indvecdf = indvecdf.groupby(['region','ind'])[['raw','after8']].sum().reset_index()
    indvecdf[['raw','after8']]=indvecdf[['raw','after8']].astype(int)
    regions = sorted(indvecdf['region'].unique().tolist())
    inds = sorted(indvecdf['ind'].unique().tolist())
    index = pd.MultiIndex.from_product([regions,inds],names=['region','ind'])
    indvecdf = pd.DataFrame(index = index).reset_index().merge(indvecdf,how='left').fillna(0).merge(indtype,how='left')
    indvecdf = indvecdf.sort_values(['region','ind']).reset_index(drop=True)
    indvecdf['exist'] = indvecdf.raw>0
    indvecdf['existafter8'] = indvecdf.after8>0
    indvecdf['growth8'] = np.log(indvecdf['after8']/indvecdf.raw)/8
    print(len(regions),len(inds))
    return indvecdf,inds

In [4]:
var = 'emp'
indvecdf,inds = load_ind_vec(var)
indvecdf.head()

927 415


/nfs/bigdata_nobackup/c/cid_postdoc/yang/conda/envs/py38/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,region,ind,raw,after8,indtype,exist,existafter8,growth8
0,10100,011,367.0,324.0,geo,True,True,-0.015577
1,10100,013,11.0,40.0,geo,True,True,0.161373
2,10100,016,5.0,3.0,geo,True,True,-0.063853
3,10100,017,0.0,0.0,geo,False,False,NaN
4,10100,018,6.0,4.0,geo,True,True,-0.050683


In [5]:
indvecdf.dtypes

region          object
ind             object
raw            float64
after8         float64
indtype         object
exist             bool
existafter8       bool
growth8        float64
dtype: object

In [ ]:
## generate 100 (20*5 fold) train-test samples from the raw data and create variables used in grid search
for i in range(20):
    kf = KFold(n_splits=5,random_state=42+i*3,shuffle=True)
    idx = kf.split(indvecdf)
    testlist = [y for x,y in idx]
    for j in range(5):
        train = indvecdf.copy()
        train['rawraw'] = train.raw
        train['testidx'] = False
        train.loc[testlist[j], "testidx"] = True
        train.loc[testlist[j], "raw"] = 0
        train['bin'] = np.where(train.raw>0,1,0)
        train['regionsum'] = train.groupby('region')['raw'].transform('sum')
        train['indsum'] = train.groupby('ind')['raw'].transform('sum')
        train['regionsum'] = np.where(train['regionsum']>1,train['regionsum'],1)
        train['indsum'] = np.where(train['indsum']>1,train['indsum'],1)
        total = train['raw'].sum()
        train['rca'] = train['raw']*total/train['regionsum']/train['indsum']
        train['rca2'] = train.rca/(train.rca+1)
        train['bin_rca'] = np.where(train.rca>1,1,0)
        train['regionsum'] = np.log(train['regionsum'])
        train['indsum'] = np.log(train['indsum'])
        train['lograw'] = np.log(train.raw+1)
        train['lograwraw'] = np.log(train.rawraw+1)
        train['pmi'] = train.lograw + np.log(total) - train.regionsum - train.indsum
        train['ppmi'] = np.where(train.rca>1,np.log(train.rca),0)
        algorithm = pyhdfe.create(train[['region','ind']],drop_singletons=False)
        residualized = algorithm.residualize(train['lograw'].values.reshape(-1, 1))
        train['feresid'] = residualized
        train['fepred'] = train['lograw'] - train['feresid']
        train['intercept'] = 1.0
        reg = LinearRegression().fit(train[['regionsum','indsum']], train.lograw)
        train['bslpred'] = reg.predict(train[['regionsum','indsum']])
        train['resid'] = train['lograw'] - train['bslpred']
        pos = smf.glm('raw~regionsum+indsum', data=train,family=sm.families.Poisson()).fit(disp=0)
        train['pospred'] = np.log(pos.predict()+1)
        train['posresid'] = train['lograw'] - train['pospred']
        train['bin_feresid'] = np.where(train.feresid>0,1,0)
        train['bin_resid'] = np.where(train.resid>0,1,0)
        train['bin_posresid'] = np.where(train.posresid>0,1,0)
        train.to_parquet(f'data/sample{i*5+j}.parquet',index=False,compression='gzip')
        del algorithm,residualized,reg,pos,train,total

In [8]:
sample = pd.read_parquet(f'data/sample99.parquet')
sample.columns

Index(['region', 'ind', 'raw', 'after8', 'indtype', 'exist', 'existafter8',
       'growth8', 'rawraw', 'testidx', 'bin', 'regionsum', 'indsum', 'rca',
       'rca2', 'bin_rca', 'lograw', 'lograwraw', 'pmi', 'ppmi', 'feresid',
       'fepred', 'intercept', 'bslpred', 'resid', 'pospred', 'posresid',
       'bin_feresid', 'bin_resid', 'bin_posresid'],
      dtype='object')